In [41]:
!which python

/home/wangling/miniconda3/envs/p4ds-env/bin/python


# Final project: Phân tích dữ liệu Data Engineer Jobs
Các thành viên:

1. Phạm Thị Hoài Hiền - 18120178

2. Vương Thị Ngọc Linh - 18120195

In [42]:
import numpy as np 
import matplotlib as plt
import pandas as pd
import re

## Giới thiệu về dữ liệu

Đây là bộ dữ liệu về việc làm Data Engineer và một số vị trí liên quan, link Kaggle lấy dữ liệu: https://www.kaggle.com/andrewmvd/data-engineer-jobs

Datatset này được tạo bởi Picklesueat và nó chứa hơn 2500 công việc liên quan vị trí Data Engineer.

Dataset gồm có 15 cột:
- Job Tile: Tên vị trí
- Salary Estimate: Mức lương
- Job Description: Miêu tả vị trí
- Rating: Xếp hạng công ty tuyển dụng
- Company name: Tên công ty tuyển dụng
- Location: Địa chỉ nơi làm việc
- Headquarters: Trụ sở của công ty
- Size: Quy mô nhân lực của công ty
- Founded: Năm thành lập công ty
- Type of ownership: Quyền sở hữu
- Industry: Lĩnh vực hoạt động của công ty
- Sector: Lĩnh vực chuyên ngành mà công ty phát triển.
- Revenue: Doanh thu mỗi năm của công ty.
- Competitors: Đối thủ cạnh tranh của công ty.
- Easy Apply: Vị trí này có dễ apply không.


## Tiền xử lí

** Đọc dữ liệu vào dataframe **

In [43]:
data_engineer_jobs = pd.read_csv("DataEngineer.csv")
data_engineer_jobs.shape

(2528, 15)

Dataset có 2528 dòng và 15 cột, đúng như miêu tả.

In [44]:
data_engineer_jobs.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,Data Engineer,$80K-$150K (Glassdoor est.),Company Description\nSagence is a management a...,4.5,Sagence\n4.5,"New York, NY","Chicago, IL",1 to 50 employees,2009,Company - Private,Consulting,Business Services,$10 to $25 million (USD),"WCI Consulting, PwC",-1
1,Senior Data Engineer (Healthcare Domain experi...,$80K-$150K (Glassdoor est.),"Key Responsibilities\n\n- Architect, build, an...",3.4,Enterprise Integration\n3.4,"New York, NY","Jacksonville, FL",51 to 200 employees,1998,Company - Private,IT Services,Information Technology,$25 to $50 million (USD),-1,-1
2,Data Engineers,$80K-$150K (Glassdoor est.),Overview\n\nJob description\n\nPosition Overvi...,5.0,Maestro Technologies\n5.0,"New York, NY","Trenton, NJ",51 to 200 employees,2003,Company - Private,IT Services,Information Technology,$5 to $10 million (USD),-1,-1
3,Client Trade Support Engineer,$80K-$150K (Glassdoor est.),About the Position\n\n\nThis position will sup...,4.8,Jane Street\n4.8,"New York, NY","New York, NY",501 to 1000 employees,2000,Company - Private,Investment Banking & Asset Management,Finance,Unknown / Non-Applicable,-1,-1
4,Data Engineer,$80K-$150K (Glassdoor est.),Data Engineer\n\nJob Details\nLevel\nExperienc...,3.7,GNY Insurance Companies\n3.7,"New York, NY","New York, NY",201 to 500 employees,1914,Company - Private,Insurance Carriers,Insurance,$100 to $500 million (USD),"Travelers, Chubb, Crum & Forster",True


** Xem kiểu dữ liệu các cột **

In [45]:
data_engineer_jobs.dtypes

Job Title             object
Salary Estimate       object
Job Description       object
Rating               float64
Company Name          object
Location              object
Headquarters          object
Size                  object
Founded                int64
Type of ownership     object
Industry              object
Sector                object
Revenue               object
Competitors           object
Easy Apply            object
dtype: object

In [46]:
data_engineer_jobs.isnull().sum() / len(data_engineer_jobs) * 100

Job Title            0.0
Salary Estimate      0.0
Job Description      0.0
Rating               0.0
Company Name         0.0
Location             0.0
Headquarters         0.0
Size                 0.0
Founded              0.0
Type of ownership    0.0
Industry             0.0
Sector               0.0
Revenue              0.0
Competitors          0.0
Easy Apply           0.0
dtype: float64

Tỉ lệ missing value của tưng cột là 0.0, tuy nhiên sau khi xem dữ liệu, có thể thấy tác giả đã thay các giá trị thiếu thành -1 và Unknown / Non-Applicable

In [47]:
print(len(data_engineer_jobs[data_engineer_jobs['Rating'] == -1]) / len(data_engineer_jobs) * 100)
print(len(data_engineer_jobs[data_engineer_jobs['Size'] == '-1']) / len(data_engineer_jobs) * 100)
print(len(data_engineer_jobs[data_engineer_jobs['Founded'] == -1]) / len(data_engineer_jobs) * 100)


9.25632911392405
4.707278481012659
22.587025316455698


Cột Competitors và Easy Apply có rất nhiều missing value, và dữ liệu hai cột này cũng không cần thiết cho câu hỏi, vì vậy sẽ xóa hai cột này để làm sạch dữ liệu.

In [48]:
data_engineer_jobs = data_engineer_jobs.drop(columns=["Competitors", "Easy Apply"])
#data_engineer_jobs



** Đổi giá trị cột Revenue từ các khoảng doanh thu bằng doanh thu cao nhất trong khoảng. Các dòng có giá trị X+ sẽ được quy ước là X+1, và các dòng có giá trị Less than X và missing value sẽ được quy ước là 0 **


In [49]:
def find_max_revenue(x):
    if x == 'Unknown / Non-Applicable' or x == '-1':
        return 0
    else:
        S = [int(s) for s in re.findall(r'\d+', x)]
        if 'billion' in x and 'million' not in x:
            S = [s*1000 for s in S]
        if '+' in x:
            S.append(S[0] + 1)
        if 'Less than' in x:
            S.append(S[0])
            S[0] = 0
        return S[1]

In [50]:

data_engineer_jobs["Revenue"] = data_engineer_jobs['Revenue'].apply(lambda x: find_max_revenue(x))
#data_engineer_jobs

** Đổi giá trị của cột Salary Estimate từ mức lượng min-max thành mức lương trung bình để trả lời câu hỏi phía dưới**

In [51]:
def find_mean_salary(x):
    S = [int(s) for s in re.findall(r'\d+', x)]
    return (S[0] + S[1]) / 2

data_engineer_jobs["Salary Estimate"] = data_engineer_jobs["Salary Estimate"].apply(lambda x: find_mean_salary(x))
#data_engineer_jobs

**Bỏ phần "\n rating" phía sau tên công ty**

In [53]:
data_engineer_jobs['Company Name'] = data_engineer_jobs["Company Name"].str.partition("\n")
data_engineer_jobs

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Engineer,115.0,Company Description\nSagence is a management a...,4.5,Sagence,"New York, NY","Chicago, IL",1 to 50 employees,2009,Company - Private,Consulting,Business Services,25
1,Senior Data Engineer (Healthcare Domain experi...,115.0,"Key Responsibilities\n\n- Architect, build, an...",3.4,Enterprise Integration,"New York, NY","Jacksonville, FL",51 to 200 employees,1998,Company - Private,IT Services,Information Technology,50
2,Data Engineers,115.0,Overview\n\nJob description\n\nPosition Overvi...,5.0,Maestro Technologies,"New York, NY","Trenton, NJ",51 to 200 employees,2003,Company - Private,IT Services,Information Technology,10
3,Client Trade Support Engineer,115.0,About the Position\n\n\nThis position will sup...,4.8,Jane Street,"New York, NY","New York, NY",501 to 1000 employees,2000,Company - Private,Investment Banking & Asset Management,Finance,0
4,Data Engineer,115.0,Data Engineer\n\nJob Details\nLevel\nExperienc...,3.7,GNY Insurance Companies,"New York, NY","New York, NY",201 to 500 employees,1914,Company - Private,Insurance Carriers,Insurance,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,Cloud Engineer,102.0,The Expertise You Have Proven expertise in dev...,4.5,PMG Global,"Westlake, TX","Herndon, VA",1 to 50 employees,-1,Company - Private,Computer Hardware & Software,Information Technology,5
2524,Electrical Engineer,102.0,Lockheed Martin is seeking a Circuit and FPGA ...,2.9,Albin Engineering Services,"Fort Worth, TX","Santa Clara, CA",51 to 200 employees,1993,Company - Private,Staffing & Outsourcing,Business Services,10
2525,Sr. Big Data Engineer,102.0,Posting Title Principal Data Engineer (Contrac...,3.7,Cincinnati Bell Technology Solutions,"Irving, TX","Cincinnati, OH",501 to 1000 employees,-1,Company - Private,IT Services,Information Technology,500
2526,Senior Big Data / ETL Engineer,102.0,"Your Opportunity\n\nCharles Schwab & Co., Inc ...",3.5,Schwab,"Westlake, TX","Dettenheim, Germany",Unknown,-1,Company - Public,-1,-1,0


## Các câu hỏi cần trả lời

**1 - Mức lương trung bình của từng job là bao nhiêu? Mức lương cho Senior data là bao nhiêu?**

In [ ]:
#CODE HERE

**2 - Top 5 lĩnh vực tuyển dụng data engineer nhiều nhất?**

In [ ]:
#CODE HERE

**3 - Top 10 location tuyển dụng nhiều vị trí nhất?**

In [ ]:
#CODE HERE

**4 - Công ty nào có quy mô lớn nhất trong mỗi khu vực (quy mô: doanh thu, sl nhân viên, năm thành lập, sl job tuyển dụng). Có phải công ty quy mô càng lớn thì rating càng cao không?**

In [ ]:
#CODE HERE

**5 - Lĩnh vực nào có mức lương trung bình (trên tất cả job) cao nhất và thấp nhất?**

In [ ]:
#CODE HERE